In [ ]:
import os
import json
import json
import regex as re
from tqdm.auto import tqdm
tqdm.pandas()
import math
import pandas as pd
import string
from pyvi.ViTokenizer import tokenize
import pandas as pd
import numpy as np
import json, pickle
from rank_bm25 import BM25Okapi
import argparse
import gc
from tqdm.auto import tqdm
tqdm.pandas()
import regex as re
import string

import json 
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import string

from pandarallel import pandarallel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

In [ ]:
def get_topk(query, topk = 100):
    tokenized_query = query.split()
    tfidf_query = tfidf_model[dictionary.doc2bow(tokenized_query)]
    scores = bm25_index[tfidf_query]
    top_n = np.argsort(scores)[::-1][:topk]
    titles = [df_wiki.title.values[i] for i in top_n]
    texts = [df_wiki.text.values[i] for i in top_n]
    # print(titles)
    # print(tfidf_query, scores)
    return titles, texts, scores[top_n]

def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
def strip_answer_string(text): 
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

In [ ]:
wiki_cleaned_path = "./processed/wikipedia_20220620_cleaned_v2.csv"
test_data_path =  "./za-data/zac2022_testa_sample_submission.json"
topk = 300

In [ ]:
df_wiki = pd.read_csv(wiki_cleaned_path)

In [ ]:
df_wiki = df_wiki.fillna("NaN")
if "title" not in df_wiki.columns:
    df_wiki["title"] = df_wiki["titles="].fillna("")

In [ ]:
df_wiki['bm25_text'] = df_wiki['bm25_text'].apply(lambda x: x.lower()).parallel_apply(post_process)
corpus = [x.split() for x in df_wiki['bm25_text'].values]
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries
dictionary.save("./outputs/bm25_stage1/dict")
tfidf_model.save("./outputs/bm25_stage1/tfidf")
bm25_index.save("./outputs/bm25_stage1/bm25_index")